In [57]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import os
import re

class RestaurantCrawler:
    def __init__(self):
        # 크롬 드라이버 설정
        self.chrome_options = Options()
        self.chrome_options.add_argument("--start-maximized")
        # self.chrome_options.add_argument("--headless")  # 필요시 헤드리스 모드 활성화
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                                       options=self.chrome_options)
        self.wait = WebDriverWait(self.driver, 3)
        
        csv_path = "../../data/external/google_gangnam_crawling_data.csv"

        # 결과 저장용 데이터프레임 - 기존 CSV 파일이 있으면 로드, 없으면 빈 데이터프레임 생성
        if os.path.exists(csv_path):
            self.restaurants_df = pd.read_csv(csv_path)
            self.csv_path = csv_path
        else:
            self.restaurants_df = pd.DataFrame(columns=[
                '음식점_이름','주소','전화번호','음식점_태그','메뉴_정보',
                '카테고리','음식점_사진','위도','경도','영업시간','리뷰'
            ])
            self.csv_path = "../../data/external/google_gangnam_crawling_data.csv"

    def search_google_maps(self, search_query):
        """구글 지도에서 음식점 검색 및 정보 수집"""
        try:
            self.driver.get("https://www.google.com/maps")
            
            # 검색창 찾기 및 검색어 입력
            search_box = self.wait.until(EC.presence_of_element_located(
                (By.ID, "searchboxinput")))
            search_box.clear()
            search_box.send_keys(search_query)
            search_box.send_keys(Keys.ENTER)
            
            # 검색 결과 대기
            time.sleep(3)

            # 현재 URL 확인하여 상세 페이지인지 목록 페이지인지 판단
            current_url = self.driver.current_url
            # 여러 결과 (목록 페이지인 경우) - 첫 번째 결과 클릭
            if "/place/" not in current_url:
                # try:
                #     # 첫 번째 음식점 이름 가져오기
                #     first_result = self.driver.find_element(By.CSS_SELECTOR, "a.hfpxzc")
                #     first_result_name = first_result.get_attribute("aria-label")
                    
                #     # 첫 번째 음식점 이름으로 다시 검색
                #     search_box = self.driver.find_element(By.ID, "searchboxinput")
                #     search_box.clear()
                #     search_box.send_keys(first_result_name)
                #     search_box.send_keys(Keys.ENTER)
                #     time.sleep(4)  # 상세 페이지 로딩 대기
                # except Exception as e:
                #     print(f"첫 번째 음식점 이름으로 다시 검색 중 오류: {e}")
                try:
                    # 첫 번째 결과 요소 찾기 (여러 선택자 시도)
                    selectors = [
                        "a.hfpxzc",  # 일반적인 결과 카드 선택자
                        "div.Nv2PK",  # 결과 컨테이너
                        "div[jsaction*='mouseover']",  # 마우스오버 이벤트가 있는 요소
                        "div.THOPZb"  # 결과 항목 컨테이너
                    ]
                    
                    for selector in selectors:
                        try:
                            first_results = self.driver.find_elements(By.CSS_SELECTOR, selector)
                            if first_results:
                                # JavaScript로 직접 클릭 (더 안정적)
                                self.driver.execute_script("arguments[0].click();", first_results[0])
                                time.sleep(1)  # 사이드바 로딩 대기
                                break
                        except:
                            continue

                    # 세 번째 m6QErb.DxyBCb.kA9KIf.dS8AEf 요소가 사이드 탭임
                    sidebars = self.driver.find_elements(By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf")
                    if len(sidebars) >= 3:
                        sidebar = sidebars[2]  # 0: 검색 결과, 1: 지도, 2: 사이드 탭
                        # 사이드 탭으로 스크롤 이동
                        self.driver.execute_script("arguments[0].scrollIntoView();", sidebar)
                        time.sleep(1)
                    else:
                        print("사이드 탭을 찾을 수 없습니다.")
                except Exception as e:
                    print(f"첫 번째 결과 클릭 및 사이드 탭 탐색 중 오류: {e}")


            # 음식점 이름 (검색어 기반 기본값 포함)
            try:
                restaurant_name = self.driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob").text
                restaurant_name = re.sub(r'\s*\([^)]*\)', '', restaurant_name)
            except:
                restaurant_name = search_query

            # 카테고리 정보
            try:
                category = self.driver.find_element(
                    By.CSS_SELECTOR, "button.DkEaL ").text
            except:
                category = ""
            
            # 음식점 사진 URL
            try:
                photo_element = self.driver.find_element(
                    By.CSS_SELECTOR, "button[aria-label*='사진'] img")
                photo_url = photo_element.get_attribute("src")
            except:
                try:
                    photo_element = self.driver.find_element(
                        By.CSS_SELECTOR, ".RZ66Rb img[decoding='async']")
                    photo_url = photo_element.get_attribute("src")
                except:
                    photo_url = ""

            # 영업시간 추출 (버튼 클릭 방식)
            try:
                # 영업시간 버튼 찾기 및 클릭
                hours_button = self.driver.find_element(By.CSS_SELECTOR, "div.OMl5r[aria-expanded='false']")
                hours_button.click()
                time.sleep(1)  # 테이블이 로드될 시간 부여
                
                # 영업시간 테이블 찾기
                hours_table = self.driver.find_element(By.CSS_SELECTOR, "table.eK4R0e")
                
                # 요일별 영업시간 추출
                business_hours = {}
                rows = hours_table.find_elements(By.CSS_SELECTOR, "tr.y0skZc")
                
                for row in rows:
                    day = row.find_element(By.CSS_SELECTOR, "td.ylH6lf div").text
                    hours_list = row.find_elements(By.CSS_SELECTOR, "li.G8aQO")
                    hours_text = [hour.text for hour in hours_list]
                    business_hours[day] = hours_text
                
                # 딕셔너리 형태로 저장
                hours = business_hours
            except Exception as e:
                print(f"영업시간 추출 중 오류 발생: {e}")
                hours = {}

            latitude, longitude = self.extract_coordinates_from_google_maps()

            # 리뷰 수집
            try:
                # 모든 리뷰 수집
                all_reviews = self.get_all_reviews()
                # 딕셔너리 형태로 저장된 리뷰를 그대로 전달
                reviews_data = all_reviews
            except Exception as e:
                print(f"리뷰 수집 중 오류: {e}")
                reviews_data = []

            return {
                '음식점_이름': restaurant_name,
                '카테고리': category,
                '음식점_사진': photo_url,
                '영업시간': hours,
                '위도': latitude,
                '경도': longitude,
                '리뷰': reviews_data
            }

            
        except Exception as e:
            print(f"구글 지도 크롤링 중 오류 발생: {e}")
            return {
                '음식점_이름': search_query,
                '카테고리': "",
                '음식점_사진': "",
                '영업시간': "",
                '리뷰': "",
                '경도': "",
                '리뷰': ""
            }

    def search_diningcode(self, restaurant_name):
        """다이닝코드에서 음식점 검색 및 정보 수집"""
        try:
            restaurant_name = restaurant_name.rstrip()
            if "점" in restaurant_name:
                idx = restaurant_name.rfind("점")
                restaurant_name = restaurant_name[:idx]

            # 1. 검색 페이지 이동
            self.driver.get(f"https://www.diningcode.com/list.dc?query={restaurant_name}")
            
            # 검색 결과 대기 (명시적 대기)
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[id^='block']"))
            )

            # 첫 번째 결과 선택 (수정된 부분)
            first_result = self.driver.find_element(By.CSS_SELECTOR, "a[id^='block']")
            detail_url = first_result.get_attribute('href')
            self.driver.get(detail_url)
            
            # 6. 주소 (도로명 + 상세주소)
            try:
                locat = self.driver.find_element(By.CLASS_NAME, 'locat')
                address_links = locat.find_elements(By.TAG_NAME, 'a')
                address_span = locat.find_element(By.TAG_NAME, 'span')
                road_address = ' '.join([a.text for a in address_links]) + address_span.text.strip()
                # 공백 정규화
                road_address = ' '.join(road_address.split())
            except:
                road_address = ""

            # 7. 전화번호
            try:
                tel = self.driver.find_element(By.CLASS_NAME, 'tel')
                phone = tel.text.strip()
                print(phone)
            except:
                phone = ""

            # 8. 메뉴 정보
            """메뉴 정보 수집 (스크롤 & 더보기 버튼 클릭 포함)"""
            menu_list = []
            try:
                # 1. 메뉴 섹션으로 스크롤
                menu_section = self.wait.until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.menu-info"))
                )
                self.driver.execute_script("arguments[0].scrollIntoView();", menu_section)
                time.sleep(1)

                # 2. "더보기" 버튼 클릭 시도
                try:
                    # 1. 모든 "더보기" 버튼 찾기
                    more_buttons = self.driver.find_elements(By.CSS_SELECTOR, "a.more-btn")
                    
                    # 2. 두 번째 버튼 클릭 (인덱스 1)
                    if len(more_buttons) > 1:
                        # 명시적 대기 + 스크롤 조정
                        more_btn = self.wait.until(
                            EC.element_to_be_clickable((more_buttons[1]))
                        )
                        self.driver.execute_script("arguments[0].scrollIntoView();", more_btn)
                        more_btn.click()
                        time.sleep(0.3)
                except Exception as e:
                    print(f"더보기 버튼 클릭 실패: {e}")
                    pass  # 버튼이 없거나 클릭 불가시 무시

                # 3. 모든 메뉴 항목 추출
                menu_items = [item for item in self.driver.find_elements(
                    By.CSS_SELECTOR, "ul.Restaurant_MenuList li"
                ) if item.is_displayed()]

                for item in menu_items:
                    try:
                        name = item.find_element(By.CSS_SELECTOR, "span.Restaurant_Menu").text
                        price = item.find_element(By.CSS_SELECTOR, "p.r-txt").text
                        menu_list.append({
                            'menu_name': name,
                            'menu_price': price if price else ""
                        })
                    except:
                        continue

            except Exception as e:
                print(f"메뉴 수집 오류: {e}")
            
            # 9. 음식점 태그 (분위기만 수집)
            """음식점 태그 정보 수집 - 분위기 카테고리만 수집"""
            tags_list = []
            try:
                # 1. 태그 섹션으로 스크롤
                tag_section = self.wait.until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "ul.app-arti"))
                )
                self.driver.execute_script("arguments[0].scrollIntoView();", tag_section)
                time.sleep(1)
                
                # 2. 분위기 카테고리 찾기
                try:
                    mood_category = self.driver.find_element(
                        By.XPATH, "//ul[@class='app-arti']/li[contains(span[@class='btxt'], '분위기')]"
                    )
                    
                    # 분위기 카테고리에 더보기 버튼이 있으면 클릭
                    try:
                        more_btn = mood_category.find_element(By.CSS_SELECTOR, "span.more-btn.button")
                        if more_btn.is_displayed():
                            self.driver.execute_script("arguments[0].scrollIntoView();", more_btn)
                            more_btn.click()
                            time.sleep(0.3)
                    except:
                        pass
                    
                    # 분위기 태그들 추출 (더보기 버튼 제외)
                    tag_elements = mood_category.find_elements(By.CSS_SELECTOR, "span.icon")
                    
                    for tag in tag_elements:
                        if tag.is_displayed() and "more-btn" not in tag.get_attribute("class"):
                            tag_text = tag.text.strip()
                            # 태그에서 이름과 카운트 분리
                            if "(" in tag_text and ")" in tag_text:
                                tag_name = tag_text[:tag_text.rfind("(")].strip()
                                count_str = tag_text[tag_text.rfind("(")+1:tag_text.rfind(")")].strip()
                                try:
                                    count = int(count_str)
                                except:
                                    count = 0
                                
                                tags_list.append({
                                    'tags': tag_name,
                                    'count': count
                                })
                
                except Exception as e:
                    print(f"분위기 태그 처리 오류: {e}")

            except Exception as e:
                print(f"태그 수집 오류: {e}")
            
            
            # 10. 결과 딕셔너리 반환
            return {
                '주소': road_address,
                '전화번호': phone,
                '메뉴_정보': menu_list,
                '음식점_태그': tags_list
            }
            
        except Exception as e:
            print(f"다이닝코드 크롤링 오류: {e}")
            return {
                '주소': "",
                '전화번호': "",
                '메뉴_정보': "",
                '음식점_태그': ""
            }
        
    def extract_coordinates_from_google_maps(self):
        """구글 맵스에서 좌표 정보 추출 (Set 사용하여 중복 제거)"""
        try:
            # 현재 URL 가져오기
            current_url = self.driver.current_url
            
            # URL에서 좌표 추출 패턴
            patterns = [
                r"!3d([-\d\.]+)!4d([-\d\.]+)",  # 패턴 1: !3d<위도>!4d<경도>
                r"@([-\d\.]+),([-\d\.]+)"       # 패턴 2: @<위도>,<경도>
            ]
            
            # 모든 패턴으로 검색하여 좌표 추출
            coordinates = set()
            for pattern in patterns:
                match = re.search(pattern, current_url)
                if match:
                    latitude = float(match.group(1))
                    longitude = float(match.group(2))
                    coordinates.add((latitude, longitude))
            
            # 좌표가 추출되었으면 첫 번째 좌표 반환
            if coordinates:
                latitude, longitude = next(iter(coordinates))
                return latitude, longitude
            
            return None, None
        except Exception as e:
            print(f"좌표 추출 중 오류: {e}")
            return None, None


    def get_all_reviews(self):
        """모든 리뷰 데이터를 딕셔너리 형태로 수집"""
        reviews = []
        try:
            # 리뷰 섹션으로 이동 (개선된 버전)
            try:                
                # 리뷰 탭 클릭 (다국어 대응)
                review_tab = self.wait.until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR, '[role="tab"][aria-label*="리뷰"], [role="tab"][aria-label*="Reviews"]')
                    )
                )
                self.driver.execute_script("arguments[0].click();", review_tab)
                time.sleep(1)  # 리뷰 컨텐츠 로딩 대기 시간 증가
                
            except Exception as e:
                print(f"리뷰 탭 클릭 실패: {e}")
                return []

            # 리뷰 컨테이너 찾기
            selectors = [
                "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde",
                "div.m6QErb.XiKgde",
                "div.m6QErb",
                "div[role='feed']",
                "div[aria-label*='결과']",  # 결과 컨테이너 (언어 무관)
                "//div[contains(@class, 'section-scrollbox')]",
                "//div[contains(@class, 'm6QErb')]"
            ]

            scroll_container = None
            for selector in selectors:
                try:
                    if selector.startswith("//"):
                        scroll_container = self.driver.find_element(By.XPATH, selector)
                    else:
                        scroll_container = self.driver.find_element(By.CSS_SELECTOR, selector)
                    break
                except:
                    continue

            if scroll_container is None:
                raise Exception("리뷰 컨테이너를 찾을 수 없습니다.")

            # 이미 수집한 리뷰 ID를 저장할 집합
            collected_reviews = set()
            scroll_attempts = 0
            no_new_review_attempts = 0
            max_scroll_attempts = 50
            max_no_new_review_attempts = 3

            while scroll_attempts < max_scroll_attempts:
                # 현재 리뷰 수 확인
                review_elements = self.driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")
                
                # 다양한 스크롤 방식 시도
                try:
                    # 방법 1: scrollBy 사용
                    self.driver.execute_script(
                        "arguments[0].scrollTop = arguments[0].scrollHeight", 
                        scroll_container
                    )
                except:
                    try:
                        # 방법 2: 키보드 방향키 사용
                        actions = ActionChains(self.driver)
                        actions.move_to_element(scroll_container).send_keys(Keys.PAGE_DOWN).perform()
                    except:
                        # 방법 3: JavaScript scrollTo 사용
                        self.driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll_container)

                # 리뷰 요소 수집
                review_elements = self.driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")
                new_reviews_found = False

                for review in review_elements:
                    try:
                        # "더보기" 버튼 찾아서 클릭
                        try:
                            more_button = review.find_element(By.CSS_SELECTOR, "button.w8nwRe.kyuRq")
                            self.driver.execute_script("arguments[0].click();", more_button)
                            time.sleep(0.3)  # 내용이 펼쳐질 시간 부여
                        except:
                            pass  # 더보기 버튼이 없는 경우 무시
                        
                        reviewer = review.find_element(By.CSS_SELECTOR, "div.d4r55").text
                        content = review.find_element(By.CSS_SELECTOR, "span.wiI7pd").text
                        review_id = f"{reviewer}:{content[:30]}"

                        try:
                            # 리뷰 사진 컨테이너 찾기
                            photo_container = review.find_element(By.CSS_SELECTOR, "div.KtCyie")
                            # 사진 버튼들 모두 찾기
                            photo_buttons = photo_container.find_elements(By.CSS_SELECTOR, "button.Tya61d")
                            # background-image URL 추출 (정규식 사용)
                            import re
                            urls = []
                            for btn in photo_buttons[:2]:  # 최대 2장만
                                style = btn.get_attribute("style")
                                if "background-image: url(" in style:
                                    url = re.search(r'url\("([^"]+)"\)', style).group(1)
                                    urls.append(url)
                            photos = urls
                        except Exception as e:
                            photos = []


                        if review_id not in collected_reviews:
                            reviews.append({
                                'reviewer': reviewer,
                                'content': content,
                                'photo': photos
                            })
                            collected_reviews.add(review_id)
                            new_reviews_found = True
                    except Exception as e:
                        print(f"리뷰 처리 중 오류: {e}")
                        continue

                if not new_reviews_found:
                    no_new_review_attempts += 1
                    if no_new_review_attempts >= max_no_new_review_attempts:
                        print(f"더 이상 새 리뷰가 로드되지 않아 스크롤 종료 (연속 {max_no_new_review_attempts}회)")
                        break
                else:
                    no_new_review_attempts = 0

                if len(reviews) >= 100:
                    print("리뷰 100개 수집 완료.")
                    break

        except Exception as e:
            print(f"리뷰 수집 중 오류 발생: {e}")

        return reviews
        
    def crawl_restaurant(self, search_query):
        """개별 음식점 수집 및 즉시 저장"""
        local = " 강남구"
        try:
            # 정보 수집
            google_info = self.search_google_maps(search_query+local)
            dining_info = self.search_diningcode(google_info['음식점_이름']+local)
            restaurant_info = {**dining_info, **google_info}
            
            # 데이터프레임에 추가
            self.restaurants_df = pd.concat([
                self.restaurants_df, 
                pd.DataFrame([restaurant_info])
            ], ignore_index=True)
            
            return restaurant_info
        
        except Exception as e:
            print(f"{search_query} 수집 실패: {e}")
            return None

    
    def save_to_csv(self, filename=None):
        """수집한 데이터를 CSV 파일로 저장 (중복 방지 버전)"""
        save_path = filename or self.csv_path
        
        # 현재 음식점 정보만 추출 (마지막 행)
        current_data = self.restaurants_df.iloc[[-1]]
        
        # 파일 존재 여부에 따라 모드 결정
        if os.path.exists(save_path):
            # 기존 파일에 추가 (헤더 없음)
            current_data.to_csv(
                save_path, mode='a', 
                header=False, index=False, 
                encoding='utf-8-sig'
            )
        else:
            # 새 파일 생성 (헤더 포함)
            current_data.to_csv(
                save_path, mode='w', 
                header=True, index=False, 
                encoding='utf-8-sig'
            )
        
        print(f"데이터가 {save_path}에 저장되었습니다.")

    
    def close(self):
        """드라이버 종료"""
        self.driver.quit()

# 사용 예시
if __name__ == "__main__":
    # CSV 파일에서 음식점 목록 로드
    restaurants_df = pd.read_csv('../../data/interim/gangnam_restaurants_cleaned.csv')
    
    # 시작 인덱스 설정
    start_idx = 9
    
    # 크롤러 초기화
    crawler = RestaurantCrawler()
    
    try:
        # 모든 음식점 정보 순차적으로 수집
        for i in range(start_idx, len(restaurants_df)):
            name = restaurants_df.loc[i, '음식점명']
            print('*' * 50)
            print(f'{i}번째 음식점: {name} 수집 시작')
            
            try:
                # 음식점 정보 수집
                crawler.crawl_restaurant(name)
                
                # 매 음식점마다 CSV 파일에 저장
                crawler.save_to_csv()
                print(f'{i}번째 음식점: {name} 저장 완료')
                
            except Exception as e:
                print(f'오류 발생: {e} → {i}번째 음식점: {name} 수집 실패')
            
        print("모든 음식점 크롤링 완료!")
        
    finally:
        # 드라이버 종료
        crawler.close()

**************************************************
9번째 음식점: 삼해정 수집 시작
더 이상 새 리뷰가 로드되지 않아 스크롤 종료 (연속 3회)
0507-1351-7567
데이터가 ../../data/external/google_gangnam_crawling_data.csv에 저장되었습니다.
9번째 음식점: 삼해정 저장 완료
**************************************************
10번째 음식점: 고향집 수집 시작


KeyboardInterrupt: 